In [1]:
import pandas as pd
import numpy as np
import time
import sys

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression  
from sklearn import svm

os_env = 'mac'

if os_env == 'window':
    module_path='/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Cohort_selection/'
else:
    module_path = '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Cohort_selection/'

if module_path not in sys.path:
    sys.path.append(module_path)

from cohort_loader_new_version0229 import *
import eicu_year_process

if os_env == 'window':
    module_path='/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Experiment(Supervised Learning)/evaluation/'
else:
    module_path = '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Experiment(Supervised Learning)/evaluation/'

if module_path not in sys.path:
    sys.path.append(module_path)

import get_hospital_eicu

import split
from imp import reload
reload(split)

import Multiclass_evaluation
reload(Multiclass_evaluation)

<module 'Multiclass_evaluation' from '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Experiment(Supervised Learning)/evaluation/Multiclass_evaluation.py'>

In [2]:
if os_env == 'window':
    mimic_path = '/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Case Labeling/mimic_analysis(new_version0313).csv.gz'
    eicu_path = '/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Case Labeling/eicu_analysis(new_version0313).csv.gz'

else:
    mimic_path = '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Case Labeling/mimic_analysis(new_version0313).csv.gz'
    eicu_path = '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Case Labeling/eicu_analysis(new_version0313).csv.gz'

In [3]:
mimic = pd.read_csv(mimic_path, compression = 'gzip')
mimic = mimic.drop('Shock_next_12h', axis = 1)

eicu = pd.read_csv(eicu_path, compression = 'gzip')
eicu = eicu.drop('Shock_next_12h', axis = 1)

dataset_mimic = mimic[~(mimic['gender']==2)].reset_index(drop=True)

dataset_mimic.replace([np.inf, -np.inf], np.nan, inplace=True)
dataset_mimic.fillna(0, inplace=True) 
dataset_mimic = dataset_mimic[~(dataset_mimic['Case']=='event')]
dataset_mimic = dataset_mimic[~((dataset_mimic['INDEX']=='CASE3_CASE4_DF')&(dataset_mimic['Annotation']=='no_circ'))]

eventset = mimic[(mimic['Case']=='event')].reset_index(drop=True)
dataset_mimic['Case'] = pd.to_numeric(dataset_mimic['Case'], errors='coerce')

mimic_train, mimic_valid = data_split(dataset_mimic, 9040, 0.9, Threshold=0.05, n_trial=1, mode = 'mimic')

mimic_train = mimic_train[mimic_train['INDEX']=='CASE3_CASE4_DF'].reset_index(drop=True)
mimic_valid = mimic_valid[mimic_valid['INDEX']=='CASE3_CASE4_DF'].reset_index(drop=True)

eicu_t = eicu_year_process.matching_patient(eicu)
dataset_eicu = eicu_t[~(eicu_t['gender']==2)].reset_index(drop=True)
dataset_eicu.replace([np.inf, -np.inf], np.nan, inplace=True)
dataset_eicu.fillna(0, inplace=True) 

eicu_test = dataset_eicu[~(dataset_eicu['Case']=='event')]
event_eicu = dataset_eicu[(dataset_eicu['Case']=='event')]
eicu_test = eicu_test[~((eicu_test['INDEX']=='CASE3_CASE4_DF')&(eicu_test['Annotation']=='no_circ'))]
eicu_test = eicu_test[eicu_test['INDEX']=='CASE3_CASE4_DF'].reset_index(drop=True)
eicu_test['Case'] = pd.to_numeric(eicu_test['Case'], errors='coerce')

Trial:  0
Threshold 조정 + 0.05, 현재 한계값: 0.1
========== 데이터셋 분할 정보 ==========
데이터셋 비율: 학습 = 0.90, 테스트 = 0.10
학습셋 클래스 비율: Case
1.0    174346
2.0     27608
3.0     70171
4.0     71699
Name: count, dtype: int64
테스트셋 클래스 비율: Case
1.0    17664
2.0     3030
3.0     9456
4.0     8278
Name: count, dtype: int64
--------------------------------------
========== 클래스 비율 ==========
학습셋 클래스 비율: 0.51:0.08:0.20:0.21
테스트셋 클래스 비율: 0.46:0.08:0.25:0.22
--------------------------------------
========== 환자 및 체류 정보 ==========
학습셋 환자 수: 5323
테스트셋 환자 수: 601
학습셋 체류 수: 5433
테스트셋 체류 수: 604
--------------------------------------
========== 실험 설정 ==========
분할 시드: 9040
학습 비율: 0.9
임계값: 0.1
--------------------------------------
========== 실행 결과 ==========
총 소요 시간(초): 0.26
시도한 시행 횟수: 0


In [4]:
if os_env == 'window':
    module_path='/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Experiment(Supervised Learning)/sub task prediction/'
else:
    module_path = '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Experiment(Supervised Learning)/sub task prediction'

if module_path not in sys.path:
    sys.path.append(module_path)

import incorperate_subtask_label
reload(incorperate_subtask_label)

<module 'incorperate_subtask_label' from '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Experiment(Supervised Learning)/sub task prediction/incorperate_subtask_label.py'>

In [5]:
# train, val -> Trainset (underfitting 방지)

mort_mimic_train, mort_mimic_valid, mort_eicu_test = incorperate_subtask_label.mortality_prediction_DATA(mimic_train, mimic_valid, eicu_test, os_env='mac')
mimic_mort_trainset = pd.concat([mort_mimic_train, mort_mimic_valid], axis = 0)

los_mimic_train, los_mimic_valid, los_eicu_test = incorperate_subtask_label.LOS_prediction_DATA(mimic_train, mimic_valid, eicu_test, os_env='mac')
mimic_los_trainset = pd.concat([los_mimic_train, los_mimic_valid], axis = 0)

ards8h_mimic_train, ards8h_mimic_valid, ards8h_eicu_test = incorperate_subtask_label.ARDS8_prediction_DATA(mimic_train, mimic_valid, eicu_test, os_env='mac')
mimic_ards8h_trainset = pd.concat([ards8h_mimic_train, ards8h_mimic_valid], axis = 0)

sic8h_mimic_train, sic8h_mimic_valid, sic8h_eicu_test = incorperate_subtask_label.SIC8_prediction_DATA(mimic_train, mimic_valid, eicu_test, os_env='mac')
mimic_sic8h__trainset = pd.concat([sic8h_mimic_train, sic8h_mimic_valid], axis = 0)

In [80]:
import get_hospital_eicu

#Mortality eicu test set stat...-----------------------------------------------------------------------------------------------------------------

view_mort = get_hospital_eicu.eicu_subgroup(mort_eicu_test)

mort_event_stat = view_mort[view_mort['death']=='event'].groupby('unittype').count()['death'].to_dict()
mort_obs_stat = view_mort.groupby('unittype').count()['Time_since_ICU_admission'].to_dict()
summary_mort = pd.DataFrame(list(zip(mort_event_stat.values(), mort_obs_stat.values())), index=mort_event_stat.keys(), columns=['N_evt', 'N_obs'])
summary_mort['N_obs(%)'] = summary_mort['N_obs'] / summary_mort['N_obs'].sum()

summary_mort.to_csv('subtask_testset_mort_summary.csv')

#LOS eicu test set stat...

view_los = get_hospital_eicu.eicu_subgroup(los_eicu_test)

view2 = pd.DataFrame(view_los.groupby('unittype')['remain_los'].value_counts(normalize=True)).reset_index()
view3 = pd.DataFrame(view_los.groupby('unittype').count()['Time_since_ICU_admission']).reset_index()

summary_los = pd.merge(view2, view3, how = 'inner', on=['unittype']).set_index(['unittype', 'Time_since_ICU_admission'])
summary_los.to_csv('subtask_testset_los_summary.csv')

#ARDS eicu test set stat...

view_ards = get_hospital_eicu.eicu_subgroup(ards8h_eicu_test)

ards_event_stat = view_ards[view_ards['Annotation_ARDS']=='ARDS'].groupby('unittype').count()['Annotation_ARDS'].to_dict()
ards_obs_stat = view_ards.groupby('unittype').count()['Time_since_ICU_admission'].to_dict()
summary_ards = pd.DataFrame(list(zip(ards_event_stat.values(), ards_obs_stat.values())), index=ards_event_stat.keys(), columns=['N_evt', 'N_obs'])
summary_ards['N_obs(%)'] = summary_ards['N_obs'] / summary_ards['N_obs'].sum()
summary_ards.to_csv('subtask_testset_ards_summary.csv')

#SIC eicu test set stat...

view_sic = get_hospital_eicu.eicu_subgroup(sic8h_eicu_test)

sic_event_stat = view_sic[view_sic['Annotation_SIC']=='SIC'].groupby('unittype').count()['Annotation_SIC'].to_dict()
sic_obs_stat = view_sic.groupby('unittype').count()['Time_since_ICU_admission'].to_dict()
summary_sic = pd.DataFrame(list(zip(sic_event_stat.values(), sic_obs_stat.values())), index=sic_event_stat.keys(), columns=['N_evt', 'N_obs'])
summary_sic['N_obs(%)'] = summary_sic['N_obs'] / summary_sic['N_obs'].sum()

summary_sic.to_csv('subtask_testset_sic_summary.csv') #-----------------------------------------------------------------------------------------------

In [87]:
_, mort_unittype, _ = get_hospital_eicu.make_eicu_dataset(view_mort)
_, los_unittype, _ = get_hospital_eicu.make_eicu_dataset(view_los)
_, ards_unittype, _ = get_hospital_eicu.make_eicu_dataset(view_ards)
_, sic_unittype, _ = get_hospital_eicu.make_eicu_dataset(view_sic)

In [148]:
ARDS8h_event_train = mimic_ards8h_trainset[mimic_ards8h_trainset['Annotation_ARDS']=='ARDS']
ARDS8h_event_train['ARDS_next_8h']='event'

ARDS8h_event_test = ards8h_eicu_test[ards8h_eicu_test['Annotation_ARDS']=='ARDS']
ARDS8h_event_test['ARDS_next_8h']='event'

SIC8h_event_train = mimic_sic8h__trainset[mimic_sic8h__trainset['Annotation_SIC']=='SIC']
SIC8h_event_train['SIC_next_8h']='event'

SIC8h_event_test = sic8h_eicu_test[sic8h_eicu_test['Annotation_SIC']=='SIC']
SIC8h_event_test['SIC_next_8h']='event'

In [93]:
X_train_mort, y_train_mort, _ = split.split_X_Y_MORT(mimic_mort_trainset[~(mimic_mort_trainset['death']=='event')], mode = 'mimic')
X_train_los, y_train_los, _ = split.split_X_Y_LOS(mimic_los_trainset, mode = 'mimic')
X_train_ards8h, y_train_ards8h, _ = split.split_X_Y_ARDS8h(mimic_ards8h_trainset, mode = 'mimic')
X_train_sic8h, y_train_sic8h, _ = split.split_X_Y_SIC8h(mimic_sic8h__trainset, mode = 'mimic')

In [281]:
reload(Multiclass_evaluation) 

<module 'Multiclass_evaluation' from '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Experiment(Supervised Learning)/evaluation/Multiclass_evaluation.py'>

In [282]:
#mortality

mort_models = Multiclass_evaluation.create_subtask(X_train_mort, y_train_mort)
mort_evaluation_result = Multiclass_evaluation.unit_mort_evaluation(mort_unittype, mort_models)

[Starting eICU-Test]
eICU-UnitType...


Model     AUROC     AUPRC
Subpopulation eICU Type                                   
UnitType      MICU                 xgb  0.428434  0.336412
              MICU                lgbm  0.422846  0.296926
              MICU            catboost  0.394907  0.282814
              MICU                  rf  0.461733  0.360150
              MICU                  lr  0.500672  0.392044
              MICU          naivebayes  0.549266  0.373243
              MICU                 knn  0.451535  0.380406
              Neuro ICU            xgb  0.606553  0.374765
              Neuro ICU           lgbm  0.487171  0.347275
              Neuro ICU       catboost  0.550234  0.292695
              Neuro ICU             rf  0.507072  0.279336
              Neuro ICU             lr  0.289457  0.215613
              Neuro ICU     naivebayes  0.493283  0.248552
              Neuro ICU            knn  0.605930  0.425728
              Med-Surg ICU         xgb  0.450575  0.310588
              Med-Surg ICU        lgbm  0.477838  0.319029
              Med-Surg ICU    catboost  0.440665  0.298211
              Med-Surg ICU          rf  0.526991  0.356416
              Med-Surg ICU          lr  0.419136  0.351897
              Med-Surg ICU  naivebayes  0.541411  0.379236
              Med-Surg ICU         knn  0.507199  0.402515
              Cardiac ICU          xgb  0.585181  0.465146
              Cardiac ICU         lgbm  0.582420  0.410995
              Cardiac ICU     catboost  0.596855  0.479240
              Cardiac ICU           rf  0.538227  0.359280
              Cardiac ICU           lr  0.472731  0.326766
              Cardiac ICU   naivebayes  0.556793  0.385447
              Cardiac ICU          knn  0.527982  0.379398
              SICU                 xgb  0.440891  0.297654
              SICU                lgbm  0.499219  0.337809
              SICU            catboost  0.491614  0.322641
              SICU                  rf  0.483531  0.343714
              SICU                  lr  0.461219  0.388656
              SICU          naivebayes  0.521694  0.433838
              SICU                 knn  0.514305  0.402897
              CTICU                xgb  0.526085  0.325733
              CTICU               lgbm  0.515873  0.345978
              CTICU           catboost  0.510875  0.334837
              CTICU                 rf  0.480164  0.315027
              CTICU                 lr  0.564296  0.426129
              CTICU         naivebayes  0.598222  0.485649
              CTICU                knn  0.546135  0.447249
              CSICU                xgb  0.508029  0.292377
              CSICU               lgbm  0.441977  0.276210
              CSICU           catboost  0.443718  0.254218
              CSICU                 rf  0.455717  0.265498
              CSICU                 lr  0.587822  0.414040
              CSICU         naivebayes  0.638390  0.419141
              CSICU                knn  0.545558  0.406853
              CCU-CTICU            xgb  0.417211  0.328448
              CCU-CTICU           lgbm  0.465657  0.346018
              CCU-CTICU       catboost  0.467943  0.351214
              CCU-CTICU             rf  0.492358  0.376699
              CCU-CTICU             lr  0.399276  0.352343
              CCU-CTICU     naivebayes  0.452328  0.329398
              CCU-CTICU            knn  0.566235  0.429525

In [288]:
mort_result_summary = pd.DataFrame()
result = mort_evaluation_result.reset_index()
for unittype in result['eICU Type'].unique():
    unittype_group = result[result['eICU Type'] == unittype]
    for metric in ['AUROC', 'AUPRC']:
        data = unittype_group[metric]
        mean, ci_lower, ci_upper = Multiclass_evaluation.mean_confidence_interval(data)
        summary = pd.DataFrame([{
            'UnitType': unittype,
            'Metric': metric,
            'Mean': mean,
            '95% CI Lower': ci_lower,
            '95% CI Upper': ci_upper
        }])

        mort_result_summary = pd.concat([mort_result_summary, summary])

# 결과 출력을 위해 필요한 컬럼 순서 조정
mort_result_summary = mort_result_summary[['UnitType', 'Metric', 'Mean', '95% CI Lower', '95% CI Upper']]

display(mort_result_summary.set_index(['UnitType', 'Metric']).transpose())

UnitType          MICU           Neuro ICU           Med-Surg ICU            \
Metric           AUROC     AUPRC     AUROC     AUPRC        AUROC     AUPRC   
Mean          0.458485  0.345999  0.505672  0.311995     0.480545  0.345413   
95% CI Lower  0.410288  0.306873  0.406162  0.243538     0.437763  0.310187   
95% CI Upper  0.506682  0.385126  0.605181  0.380452     0.523327  0.380639   

UnitType     Cardiac ICU                SICU               CTICU            \
Metric             AUROC     AUPRC     AUROC     AUPRC     AUROC     AUPRC   
Mean            0.551455  0.400896  0.487496  0.361030  0.534521  0.382943   
95% CI Lower    0.511660  0.349825  0.461010  0.316122  0.498678  0.319635   
95% CI Upper    0.591251  0.451967  0.513982  0.405937  0.570364  0.446252   

UnitType         CSICU           CCU-CTICU            
Metric           AUROC     AUPRC     AUROC     AUPRC  
Mean          0.517316  0.332620  0.465858  0.359092  
95% CI Lower  0.446276  0.261903  0.415534  0.326678  
95% CI Upper  0.588356  0.403336  0.516182  0.391506

In [274]:
#los

los_models = Multiclass_evaluation.create_subtask(X_train_los, y_train_los)
los_evaluation_result = Multiclass_evaluation.unit_los_evaluation(los_unittype, los_models)

[Starting eICU-Test]
eICU-UnitType...


Model  AUROC < 5days  AUROC < 10days  \
Subpopulation eICU Type                                                 
UnitType      MICU                 xgb       0.709810        0.573920   
              MICU                lgbm       0.732420        0.609771   
              MICU            catboost       0.720620        0.588533   
              MICU                  rf       0.726193        0.601428   
              MICU                  lr       0.570747        0.405825   
              MICU          naivebayes       0.665183        0.579075   
              MICU                 knn       0.723026        0.597750   
              Neuro ICU            xgb       0.821590        0.496870   
              Neuro ICU           lgbm       0.896201        0.593733   
              Neuro ICU       catboost       0.885739        0.625644   
              Neuro ICU             rf       0.885728        0.628522   
              Neuro ICU             lr       0.576683        0.380791   
              Neuro ICU     naivebayes       0.652934        0.507255   
              Neuro ICU            knn       0.818375        0.626110   
              Med-Surg ICU         xgb       0.788586        0.607850   
              Med-Surg ICU        lgbm       0.811233        0.657609   
              Med-Surg ICU    catboost       0.801024        0.646843   
              Med-Surg ICU          rf       0.792166        0.614150   
              Med-Surg ICU          lr       0.575983        0.397088   
              Med-Surg ICU  naivebayes       0.712202        0.552545   
              Med-Surg ICU         knn       0.690617        0.540198   
              Cardiac ICU          xgb       0.791061        0.611174   
              Cardiac ICU         lgbm       0.842985        0.723702   
              Cardiac ICU     catboost       0.773600        0.667978   
              Cardiac ICU           rf       0.809750        0.641586   
              Cardiac ICU           lr       0.550791        0.353415   
              Cardiac ICU   naivebayes       0.671333        0.590954   
              Cardiac ICU          knn       0.659639        0.533570   
              SICU                 xgb       0.771778        0.501149   
              SICU                lgbm       0.800273        0.575797   
              SICU            catboost       0.816059        0.553612   
              SICU                  rf       0.793086        0.557050   
              SICU                  lr       0.588796        0.480181   
              SICU          naivebayes       0.657043        0.502475   
              SICU                 knn       0.737560        0.538116   
              CTICU                xgb       0.657001        0.688240   
              CTICU               lgbm       0.665554        0.706470   
              CTICU           catboost       0.680458        0.722970   
              CTICU                 rf       0.640435        0.662752   
              CTICU                 lr       0.519329        0.328458   
              CTICU         naivebayes       0.608338        0.625409   
              CTICU                knn       0.398945        0.517596   
              CSICU                xgb       0.678115        0.634792   
              CSICU               lgbm       0.711000        0.611612   
              CSICU           catboost       0.690779        0.610992   
              CSICU                 rf       0.699697        0.615522   
              CSICU                 lr       0.576249        0.364502   
              CSICU         naivebayes       0.653530        0.586736   
              CSICU                knn       0.624162        0.489396   
              CCU-CTICU            xgb       0.766064        0.605877   
              CCU-CTICU           lgbm       0.797746        0.696757   
              CCU-CTICU       catboost       0.783338        0.630921   
              CCU-CTICU             rf       0.786428        0.593897   
              CCU-

In [290]:
pd.set_option('display.max_columns', None)
los_result_summary = pd.DataFrame()
result = los_evaluation_result.reset_index()
for unittype in result['eICU Type'].unique():
    unittype_group = result[result['eICU Type'] == unittype]
    for metric in ['AUROC < 5days','AUROC < 10days', 'AUROC >= 15days', 'AUPRC < 5days', 'AUPRC < 10days', 'AUPRC >= 15days']:
        data = unittype_group[metric]
        mean, ci_lower, ci_upper = Multiclass_evaluation.mean_confidence_interval(data)
        summary = pd.DataFrame([{
            'UnitType': unittype,
            'Metric': metric,
            'Mean': mean,
            '95% CI Lower': ci_lower,
            '95% CI Upper': ci_upper
        }])

        los_result_summary = pd.concat([los_result_summary, summary])

# 결과 출력을 위해 필요한 컬럼 순서 조정
los_result_summary = los_result_summary[['UnitType', 'Metric', 'Mean', '95% CI Lower', '95% CI Upper']]

display(los_result_summary.set_index(['UnitType', 'Metric']).transpose())

UnitType              MICU                                               \
Metric       AUROC < 5days AUROC < 10days AUROC >= 15days AUPRC < 5days   
Mean              0.692571       0.565186        0.722675      0.813648   
95% CI Lower      0.638749       0.499169        0.631499      0.792904   
95% CI Upper      0.746394       0.631203        0.813851      0.834392   

UnitType                                        Neuro ICU                 \
Metric       AUPRC < 10days AUPRC >= 15days AUROC < 5days AUROC < 10days   
Mean               0.235276        0.420122      0.791036       0.551275   
95% CI Lower       0.196513        0.323108      0.674237       0.464465   
95% CI Upper       0.274039        0.517136      0.907835       0.638085   

UnitType                                                                   \
Metric       AUROC >= 15days AUPRC < 5days AUPRC < 10days AUPRC >= 15days   
Mean                0.756879      0.791688       0.223975        0.620137   
95% CI Lower        0.632471      0.749172       0.161834        0.503178   
95% CI Upper        0.881287      0.834205       0.286116        0.737095   

UnitType      Med-Surg ICU                                               \
Metric       AUROC < 5days AUROC < 10days AUROC >= 15days AUPRC < 5days   
Mean              0.738830       0.573755        0.764914      0.793272   
95% CI Lower      0.659646       0.491073        0.667637      0.772411   
95% CI Upper      0.818014       0.656436        0.862192      0.814134   

UnitType                                      Cardiac ICU                 \
Metric       AUPRC < 10days AUPRC >= 15days AUROC < 5days AUROC < 10days   
Mean               0.254952        0.562863      0.728451       0.588911   
95% CI Lower       0.203973        0.480306      0.632008       0.478048   
95% CI Upper       0.305932        0.645421      0.824895       0.699774   

UnitType                                                                   \
Metric       AUROC >= 15days AUPRC < 5days AUPRC < 10days AUPRC >= 15days   
Mean                0.721016      0.803552       0.231485        0.590274   
95% CI Lower        0.627764      0.763443       0.180861        0.472680   
95% CI Upper        0.814267      0.843661       0.282109        0.707867   

UnitType              SICU                                               \
Metric       AUROC < 5days AUROC < 10days AUROC >= 15days AUPRC < 5days   
Mean              0.737799       0.529769        0.767728      0.775604   
95% CI Lower      0.659540       0.497018        0.662078      0.740634   
95% CI Upper      0.816059       0.562519        0.873378      0.810574   

UnitType                                            CTICU                 \
Metric       AUPRC < 10days AUPRC >= 15days AUROC < 5days AUROC < 10days   
Mean               0.188869        0.611261      0.595723       0.607414   
95% CI Lower       0.175260        0.480917      0.501311       0.477184   
95% CI Upper       0.202477        0.741605      0.690135       0.737643   

UnitType                                                                   \
Metric       AUROC >= 15days AUPRC < 5days AUPRC < 10days AUPRC >= 15days   
Mean                0.499052      0.778547       0.354924        0.150914   
95% CI Lower        0.434209      0.743192       0.275062        0.005987   
95% CI Upper        0.563895      0.813903       0.434785        0.295840   

UnitType             CSICU                                               \
Metric       AUROC < 5days AUROC < 10days AUROC >= 15days AUPRC < 5days   
Mean              0.661933       0.559079        0.681095      0.787763   
95% CI Lower      0.617633       0.468263        0.607593      0.765117   
95% CI Upper      0.706233       0.649894        0.754596      0.810409   

UnitType                                        CCU-CTICU                 \
Metric       AUPRC < 10days AUPRC >= 15days AUROC < 5days AUROC < 10days   
Mean               0.280976        0.332572

In [296]:
#ARDS
reload(Multiclass_evaluation)
# models = Multiclass_evaluation.create_subtask(X_train_ards8h, y_train_ards8h)
ards_evaluation_result = Multiclass_evaluation.unit_ards_evaluation(ards_unittype, models, ARDS8h_event_test)

[Starting eICU-Test]
eICU-UnitType...
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivi

Model     AUPRC  ARDS score
Subpopulation eICU Type                                     
UnitType      Neuro ICU            xgb  0.768920    0.348813
              Neuro ICU           lgbm  0.625769    0.147392
              Neuro ICU       catboost  0.719009    0.242086
              Neuro ICU             rf  0.856843    0.387755
              Neuro ICU             lr  0.587585    0.635179
              Neuro ICU     naivebayes  0.866372    0.000000
              Neuro ICU            knn  0.679737    0.330724
              Cardiac ICU          xgb  0.826826    0.000000
              Cardiac ICU         lgbm  0.859634    0.136842
              Cardiac ICU     catboost  0.765398    0.068783
              Cardiac ICU           rf  0.843043    0.063107
              Cardiac ICU           lr  0.685214    0.014301
              Cardiac ICU   naivebayes  0.558308    0.000000
              Cardiac ICU          knn  0.600000    0.013198
              Med-Surg ICU         xgb  0.468002    0.168667
              Med-Surg ICU        lgbm  0.465676    0.171450
              Med-Surg ICU    catboost  0.526266    0.254192
              Med-Surg ICU          rf  0.496901    0.249015
              Med-Surg ICU          lr  0.421066    0.131766
              Med-Surg ICU  naivebayes  0.407567    0.000000
              Med-Surg ICU         knn  0.192246    0.151075
              SICU                 xgb  0.814747    0.296890
              SICU                lgbm  0.924628    0.234619
              SICU            catboost  0.841746    0.424175
              SICU                  rf  0.832653    0.254617
              SICU                  lr  0.873998    0.093357
              SICU          naivebayes  0.367454    0.000000
              SICU                 knn  0.652142    0.068241
              MICU                 xgb  0.630178    0.260449
              MICU                lgbm  0.510237    0.262626
              MICU            catboost  0.563078    0.369796
              MICU                  rf  0.496142    0.374600
              MICU                  lr  0.404481    0.220084
              MICU          naivebayes  0.347074    0.000000
              MICU                 knn  0.283651    0.187717
              CTICU                xgb  1.000000    0.558282
              CTICU               lgbm  1.000000    0.309524
              CTICU           catboost  0.860119    0.385445
              CTICU                 rf  1.000000    0.573529
              CTICU                 lr  0.904545    0.022496
              CTICU         naivebayes  1.000000    0.000000
              CTICU                knn  0.816667    0.024575
              CSICU                xgb  0.183731    0.000000
              CSICU               lgbm  0.297106    0.000000
              CSICU           catboost  0.323575    0.189811
              CSICU                 rf  0.370688    0.208221
              CSICU                 lr  0.205256    0.186596
              CSICU         naivebayes  0.299889    0.000000
              CSICU                knn  0.141741    0.169589
              CCU-CTICU            xgb  0.929404    0.154534
              CCU-CTICU           lgbm  0.951057    0.181685
              CCU-CTICU       catboost  0.942230    0.259128
              CCU-CTICU             rf  0.934121    0.294149
              CCU-CTICU             lr  0.707846    0.057612
              CCU-CTICU     naivebayes  0.728558    0.000000
              CCU-CTICU            knn  0.852000    0.058559

In [297]:
ards_result_summary = pd.DataFrame()
result = ards_evaluation_result.reset_index()
for unittype in result['eICU Type'].unique():
    unittype_group = result[result['eICU Type'] == unittype]
    for metric in ['AUPRC', 'ARDS score']:
        data = unittype_group[metric]
        mean, ci_lower, ci_upper = Multiclass_evaluation.mean_confidence_interval(data)
        summary = pd.DataFrame([{
            'UnitType': unittype,
            'Metric': metric,
            'Mean': mean,
            '95% CI Lower': ci_lower,
            '95% CI Upper': ci_upper
        }])

        ards_result_summary = pd.concat([ards_result_summary, summary])

# 결과 출력을 위해 필요한 컬럼 순서 조정
ards_result_summary = ards_result_summary[['UnitType', 'Metric', 'Mean', '95% CI Lower', '95% CI Upper']]

display(ards_result_summary.set_index(['UnitType', 'Metric']).transpose())

UnitType     Neuro ICU            Cardiac ICU            Med-Surg ICU  \
Metric           AUPRC ARDS score       AUPRC ARDS score        AUPRC   
Mean          0.729176   0.298850    0.734060   0.042319     0.425389   
95% CI Lower  0.629289   0.113878    0.621751  -0.004356     0.323082   
95% CI Upper  0.829064   0.483822    0.846370   0.088994     0.527696   

UnitType                     SICU                MICU               CTICU  \
Metric       ARDS score     AUPRC ARDS score    AUPRC ARDS score    AUPRC   
Mean           0.160881  0.758195   0.195986  0.46212   0.239325  0.94019   
95% CI Lower   0.082157  0.580796   0.058578  0.34865   0.121962  0.86732   
95% CI Upper   0.239605  0.935595   0.333393  0.57559   0.356687  1.01306   

UnitType                    CSICU            CCU-CTICU             
Metric       ARDS score     AUPRC ARDS score     AUPRC ARDS score  
Mean           0.267693  0.260284   0.107745  0.863602   0.143667  
95% CI Lower   0.033577  0.182850   0.013960  0.766776   0.041774  
95% CI Upper   0.501809  0.337717   0.201530  0.960428   0.245560

In [298]:
#SIC
reload(Multiclass_evaluation)
# sic_models = Multiclass_evaluation.create_subtask(X_train_sic8h, y_train_sic8h)
sic_evaluation_result = Multiclass_evaluation.unit_sic_evaluation(sic_unittype, sic_models, SIC8h_event_test)

[Starting eICU-Test]
eICU-UnitType...
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivisionError
ZeroDivi

Model     AUPRC  SIC score
Subpopulation eICU Type                                    
UnitType      Med-Surg ICU         xgb  0.578620   0.298955
              Med-Surg ICU        lgbm  0.578897   0.294476
              Med-Surg ICU    catboost  0.568817   0.236934
              Med-Surg ICU          rf  0.558914   0.155286
              Med-Surg ICU          lr  0.398991   0.255856
              Med-Surg ICU  naivebayes  0.434763   0.252979
              Med-Surg ICU         knn  0.325293   0.217709
              MICU                 xgb  0.571134   0.259535
              MICU                lgbm  0.629291   0.266862
              MICU            catboost  0.542935   0.224009
              MICU                  rf  0.633595   0.232637
              MICU                  lr  0.449593   0.457827
              MICU          naivebayes  0.552737   0.410962
              MICU                 knn  0.245703   0.234194
              Neuro ICU            xgb  0.501627   0.238364
              Neuro ICU           lgbm  0.400016   0.000000
              Neuro ICU       catboost  0.352543   0.237027
              Neuro ICU             rf  0.393597   0.234940
              Neuro ICU             lr  0.330968   0.436641
              Neuro ICU     naivebayes  0.470026   0.366935
              Neuro ICU            knn  0.379989   0.221843
              CCU-CTICU            xgb  0.988095   0.115044
              CCU-CTICU           lgbm  0.724194   0.000000
              CCU-CTICU       catboost  1.000000   0.000000
              CCU-CTICU             rf  0.858796   0.000000
              CCU-CTICU             lr  0.147314   0.000000
              CCU-CTICU     naivebayes  0.306696   0.039731
              CCU-CTICU            knn  1.000000   0.053498
              SICU                 xgb  0.480831   0.306982
              SICU                lgbm  0.584724   0.423059
              SICU            catboost  0.477984   0.411693
              SICU                  rf  0.558202   0.380000
              SICU                  lr  0.265722   0.262626
              SICU          naivebayes  0.360956   0.143207
              SICU                 knn  0.362763   0.344828
              CSICU                xgb  0.655845   0.240592
              CSICU               lgbm  0.578484   0.229776
              CSICU           catboost  0.653943   0.199149
              CSICU                 rf  0.546910   0.125402
              CSICU                 lr  0.351371   0.217306
              CSICU         naivebayes  0.304209   0.269014
              CSICU                knn  0.325634   0.171772
              Cardiac ICU          xgb  0.326816   0.251208
              Cardiac ICU         lgbm  0.317650   0.135417
              Cardiac ICU     catboost  0.304898   0.137203
              Cardiac ICU           rf  0.385863   0.149808
              Cardiac ICU           lr  0.298110   0.187952
              Cardiac ICU   naivebayes  0.363437   0.246575
              Cardiac ICU          knn  0.234294   0.231348
              CTICU                xgb  0.505036   0.206349
              CTICU               lgbm  0.517342   0.374486
              CTICU           catboost  0.731126   0.419355
              CTICU                 rf  0.719604   0.408072
              CTICU                 lr  0.358158   0.355789
              CTICU         naivebayes  0.262802   0.474747
              CTICU                knn  0.448233   0.479866

In [299]:
sic_result_summary = pd.DataFrame()
result = sic_evaluation_result.reset_index()
for unittype in result['eICU Type'].unique():
    unittype_group = result[result['eICU Type'] == unittype]
    for metric in ['AUPRC', 'SIC score']:
        data = unittype_group[metric]
        mean, ci_lower, ci_upper = Multiclass_evaluation.mean_confidence_interval(data)
        summary = pd.DataFrame([{
            'UnitType': unittype,
            'Metric': metric,
            'Mean': mean,
            '95% CI Lower': ci_lower,
            '95% CI Upper': ci_upper
        }])

        sic_result_summary = pd.concat([sic_result_summary, summary])

# 결과 출력을 위해 필요한 컬럼 순서 조정
sic_result_summary = sic_result_summary[['UnitType', 'Metric', 'Mean', '95% CI Lower', '95% CI Upper']]

display(sic_result_summary.set_index(['UnitType', 'Metric']).transpose())

UnitType     Med-Surg ICU                MICU           Neuro ICU            \
Metric              AUPRC SIC score     AUPRC SIC score     AUPRC SIC score   
Mean             0.492042  0.244599  0.517855  0.298004  0.404109  0.247964   
95% CI Lower     0.395670  0.199295  0.393073  0.209801  0.347383  0.121435   
95% CI Upper     0.588414  0.289904  0.642638  0.386206  0.460836  0.374493   

UnitType     CCU-CTICU                SICU               CSICU            \
Metric           AUPRC SIC score     AUPRC SIC score     AUPRC SIC score   
Mean          0.717871  0.029753  0.441598  0.324628  0.488057  0.207573   
95% CI Lower  0.391606 -0.010699  0.334409  0.233845  0.343702  0.163610   
95% CI Upper  1.044136  0.070206  0.548786  0.415411  0.632411  0.251536   

UnitType     Cardiac ICU               CTICU            
Metric             AUPRC SIC score     AUPRC SIC score  
Mean            0.318724  0.191359  0.506043  0.388381  
95% CI Lower    0.273545  0.143555  0.345509  0.302642  
95% CI Upper    0.363903  0.239163  0.666577  0.474119

In [302]:
sic_result_summary.set_index(['UnitType', 'Metric']).transpose().to_csv('baseline_method_subtask_sic.csv')
ards_result_summary.set_index(['UnitType', 'Metric']).transpose().to_csv('baseline_method_subtask_ards.csv')
los_result_summary.set_index(['UnitType', 'Metric']).transpose().to_csv('baseline_method_subtask_los.csv')
mort_result_summary.set_index(['UnitType', 'Metric']).transpose().to_csv('baseline_method_subtask_mort.csv')